# 0. Getting Started with DECE Engine

In [1]:
from cf_ml.dataset import load_diabetes_dataset
from cf_ml.model import PytorchModelManager
from cf_ml.cf_engine import CFEnginePytorch

Prepare for the dataset and the model.

In [2]:
# load the sample PIMA dataset
dataset = load_diabetes_dataset()

# set the target model 
model = None
retrain = True

# initialize the model manager with the sample dataset
model_manager = PytorchModelManager(dataset, model=model)

# (re)train the model if needed
if retrain:
    model_manager.train(verbose=False)

print("Train Accuracy: {:.3f}, Test Accuracy: {:.3f}".format(
    model_manager.evaluate('train'), model_manager.evaluate('test')))

Train Accuracy: 0.788, Test Accuracy: 0.779


In [3]:
# initialize DECE engine
engine = CFEnginePytorch(dataset, model_manager)

## Use Case-1: Basic Usage
Generate one counterfactual explanation for a target example.

In [4]:
# an example instance that contains all attribute values from a patient
example = {
    'Pregnancies': 0,
    'Glucose': 120,
    'BloodPressure': 70,
    'SkinThickness': 25,
    'BMI': 30,
    'Insulin': 80,
    'DiabetesPedigreeFunction': 0.5,
    'Age': 40,
}

In [5]:
# generate one counterfactual explanation to the example
counterfactuals = engine.generate_counterfactual_examples(example)

# log the generated counterfactual explanation in [feature_1, feature_2, ..., target_class, prediction_class]
counterfactuals.all

[1/1]  Epoch-0, time cost: 0.367s, loss: 0.001, iteration: 500, validation rate: 1.000


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Outcome_pred
0,0.0,123.0,70.0,25.0,80.0,33.1,0.584,41.0,positive,positive


## Use Case-2: Generate Counterfactual Examples for Multiple Examples
Generate counterfactual explanations for multiple examples.

In [6]:
# load 500 examples stored in a pandas.DataFrame
examples = dataset.get_train_X(preprocess=False).iloc[:500]

In [7]:
counterfactuals = engine.generate_counterfactual_examples(examples)

# log all valid counterfactual explanations
counterfactuals.valid.head()

[500/500]  Epoch-0, time cost: 0.842s, loss: 1.765, iteration: 523, validation rate: 0.996


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Outcome_pred
0,7.0,113.0,85.0,29.0,131.0,31.8,0.584,55.0,negative,negative
1,5.0,125.0,60.0,23.0,0.0,34.2,0.557,28.0,positive,positive
2,1.0,165.0,95.0,34.0,680.0,52.3,0.427,27.0,positive,positive
3,1.0,131.0,55.0,24.0,141.0,31.4,0.921,29.0,positive,positive
4,8.0,110.0,0.0,0.0,25.0,29.8,0.175,38.0,negative,negative


## Use Case-3: Generate Customized Counterfactual Examples 
Generate counterfactual explanations with constraints.

In [8]:
# settings for generating customized counterfactual explanations

# num (int): number of counterfactual explanations for each example
# k (int): the most number of changeable attributes
# changeable_attr (list): attributes that are allowed to change
# cf_range (dict): variation ranges of counterfactual explanations

setting = {
    'num': 5, 
    'changeable_attr': ['Glucose', 'BMI', 'BloodPressure'], 
    'cf_range': { 
        'BloodPressure': {
            'min': 50,
            'max': 120,
        },
        'BMI': {
            'min': 20,
            'max': 50
        }
    }
}

In [9]:
counterfactuals = engine.generate_counterfactual_examples(example, setting=setting)

# log all valid counterfactual explanations
counterfactuals.valid

[1/1]  Epoch-0, time cost: 0.764s, loss: -0.001, iteration: 500, validation rate: 1.000


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Outcome_pred
0,0.0,117.0,70.0,25.0,80.0,38.1,0.5,40.0,positive,positive
1,0.0,123.0,67.0,25.0,80.0,35.2,0.5,40.0,positive,positive
2,0.0,142.0,83.0,25.0,80.0,28.7,0.5,40.0,positive,positive
3,0.0,133.0,70.0,25.0,80.0,30.0,0.5,40.0,positive,positive
4,0.0,122.0,76.0,25.0,80.0,35.7,0.5,40.0,positive,positive
